In [7]:
import time 
import requests
import random
import pymongo

In [8]:
client = pymongo.MongoClient(host = '127.0.0.1' , port = 27017)
db = client['spider']
collection = db['lagou']

In [3]:
def get_cookie():
    headers = {
        'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Accept-encoding': 'gzip, deflate, br',
        'Accept-language': 'zh-CN,zh;q=0.9',
        'Cache-Control': 'max-age=0',
        'Upgrade-Insecure-Requests':'1',
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36',
    }
    url = 'https://www.lagou.com/jobs/list_%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90?labelWords=&fromSearch=true&suginput='
    response = requests.get(url,headers = headers)
    r = requests.utils.dict_from_cookiejar(response.cookies)
    cookies = {
        'user_trace_token':'20200925153131-85ace4b9-f600-44d7-8bc8-eed1c4004d17',
        '_ga':'GA1.2.453135409.1601019090',
        'LGUID':'20200925153131-881dcfaa-b37c-485c-8510-d8d3acdac2ca',
        'JSESSIONID':'ABAAAECABFAACEAC0012F0A551229331D5F837D3577960C',
        'WEBTJ-ID':'20201125154720-175fe5f0321631-0b10d22157783a-230346d-2073600-175fe5f032296d',
        'RECOMMEND_TIP':'true',
        'PRE_UTM':'',
        'PRE_LAND':'https%3A%2F%2Fwww.lagou.com%2F',
        'sajssdk_2015_cross_new_user':'1',
        'sensorsdata2015session':'%7B%7D',
        'Hm_lvt_4233e74dff0ae5bd0a3d81c6ccf756e6':'1606290441',
        '_gid':'GA1.2.1395412730.1606290441',
        'LGSID':'20201125154721-95d54f36-a25a-4ecc-994a-8fdc220d7444',
        'PRE_HOST':'cn.bing.com',
        'PRE_SITE':'https%3A%2F%2Fcn.bing.com%2F',
        'index_location_city':'%E5%85%A8%E5%9B%BD',
        'TG-TRACK-CODE':'index_search',
        'SEARCH_ID':'dba037bc2c6e4eaba3d9fc7a1bf2b072',
        'Hm_lpvt_4233e74dff0ae5bd0a3d81c6ccf756e6':'1606290475',                                                                'LGRID':'20201125154754-2509d411-86b3-4f8b-88a4-4ca71a263003',
    }
    cookies.update(r)

    return cookies

In [4]:
def crawl(city = '' , pn = 1 , cookies = None):
    headers = {
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Content-Length': '55',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'Origin': 'https://www.lagou.com',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36(KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36',
        'X-Anit-Forge-Code': '0',
        'X-Anit-Forge-Token': 'None',
        'X-Requested-With': 'XMLHttpRequest',
        'Referer': 'https://www.lagou.com/jobs/list_%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90?labelWords=&fromSearch=true&suginput='
    }
    params = (
        ('px', 'default'),
        ('city',city),
        ('needAddtionalResult', 'false'),
    )
    data = {"first":"true",
            'kd': '数据分析',
            'pn': pn}
    if pn>1 :
        data['first'] = "false"
    response = requests.post('https://www.lagou.com/jobs/positionAjax.json?needAddtionalResult=false',headers = headers , params = params , cookies = cookies , data = data )
    return response.json()  

In [26]:

# city_list = ["北京","上海","深圳","广州","杭州","成都","南京","武汉","西安","厦门","长沙","苏州","天津"]
city_list = ["长沙","苏州","天津"]
for city in city_list:
    print("{} start".format(city))
    for i in range(1,31):
        if (i-1) % 5 == 0:
            cookies = get_cookie()
        time.sleep(random.random() + random.randint(1,2))
        resp_json = crawl(city=city,pn = i , cookies= cookies)
        try:
            position_list = resp_json['content']['positionResult']['result']
        except:
            print(resp_json)
        if not position_list:
            print('*'*60)
            break
        collection.insert_many(position_list)
    print("{} end".format(city))

长沙 start
************************************************************
长沙 end
苏州 start
{'status': False, 'msg': '您操作太频繁,请稍后再访问', 'clientIp': '211.95.30.250', 'state': 2402}
************************************************************
苏州 end
天津 start
{'status': False, 'msg': '您操作太频繁,请稍后再访问', 'clientIp': '211.95.30.250', 'state': 2402}
************************************************************
天津 end
